In [5]:
from sqlalchemy import create_engine
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, sessionmaker
import datetime
import pandas as pd

# --- Database Model Definition ---
class Base(DeclarativeBase):
    pass

class Sale(Base):
    __tablename__ = 'sales'

    id: Mapped[int] = mapped_column(primary_key=True)
    sales_person: Mapped[str] = mapped_column()
    country: Mapped[str] = mapped_column()
    product: Mapped[str] = mapped_column()
    date: Mapped[datetime.date] = mapped_column()
    amount: Mapped[str] = mapped_column()
    boxes_shipped: Mapped[int] = mapped_column()

    def __repr__(self):
        return (f"<Sale(sales_person='{self.sales_person}', country='{self.country}', "
                f"product='{self.product}', date='{self.date}', amount={self.amount}, "
                f"boxes_shipped={self.boxes_shipped})>")

# --- Function to Create Database and Table ---
def create_database(db_name='sales_data.db'):
    engine = create_engine(f'sqlite:///{db_name}')
    Base.metadata.create_all(engine)
    print(f"Database '{db_name}' and table 'sales' created successfully (if they didn't exist).")
    return engine

# --- MODIFIED: Function to Load CSV Data (with clear-table option) ---
def load_csv_to_db(csv_filepath, engine, clear_existing_data=True):
    """
    Loads data from a CSV file into the 'sales' table.
    Optionally clears existing data before loading.

    Args:
        csv_filepath (str): The path to the CSV file.
        engine: The SQLAlchemy engine connected to the database.
        clear_existing_data (bool): If True, deletes all existing records
                                    from the 'sales' table before loading.
    """
    print(f"\nAttempting to load data from {csv_filepath}...")
    Session = sessionmaker(bind=engine)
    session = Session()

    try:
        if clear_existing_data:
            num_deleted = session.query(Sale).delete()
            session.commit()
            print(f"Cleared {num_deleted} existing records from 'sales' table.")

        df = pd.read_csv(csv_filepath)

        # IMPORTANT: Adjust column renaming based on your EXACT CSV headers
        df.rename(columns={
            'Sales Person': 'sales_person',
            'Country': 'country',
            'Product': 'product',
            'Date': 'date',
            'Amount': 'amount',
            'Boxes Shipped': 'boxes_shipped'
        }, inplace=True)

        df['date'] = pd.to_datetime(df['date']).dt.date

        sales_records = []
        for index, row in df.iterrows():
            sale = Sale(
                sales_person=row['sales_person'],
                country=row['country'],
                product=row['product'],
                date=row['date'],
                amount=row['amount'],
                boxes_shipped=row['boxes_shipped']
            )
            sales_records.append(sale)

        session.add_all(sales_records)
        session.commit()
        print(f"Successfully loaded {len(sales_records)} records from '{csv_filepath}' into the 'sales' table.")

    except FileNotFoundError:
        print(f"Error: CSV file not found at '{csv_filepath}'. Please ensure the path is correct.")
    except KeyError as e:
        session.rollback() # Rollback if column issue
        print(f"Error: Missing or incorrect column name in CSV after renaming: {e}. Check your CSV headers and the `df.rename` mapping.")
    except Exception as e:
        session.rollback() # Rollback changes if any other error occurs
        print(f"An unexpected error occurred during CSV loading: {e}")
    finally:
        session.close()

# --- Main execution block for Jupyter Notebook ---

db_name = 'sales_data.db'
csv_file_path = 'C:/Users/USER/Jupyter files/Sales Analysis project/Chocolate Sales.csv'

# 1. Create the database and table, and GET the engine
engine = create_database(db_name)

# 2. Load data from CSV, clearing existing data each time
load_csv_to_db(csv_file_path, engine, clear_existing_data=True) # Set to False if you want to add duplicates

# 3. Verify data
Session = sessionmaker(bind=engine)
session = Session()
print("\n--- Current Data in Database ---")
all_sales = session.query(Sale).all()
if all_sales:
    for sale in all_sales:
        print(sale)
else:
    print("No sales data found in the database.")
session.close()

Database 'sales_data.db' and table 'sales' created successfully (if they didn't exist).

Attempting to load data from C:/Users/USER/Jupyter files/Sales Analysis project/Chocolate Sales.csv...
Cleared 0 existing records from 'sales' table.


C:\Users\USER\AppData\Local\Temp\ipykernel_6124\4257172290.py:67: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date']).dt.date


Successfully loaded 1094 records from 'C:/Users/USER/Jupyter files/Sales Analysis project/Chocolate Sales.csv' into the 'sales' table.

--- Current Data in Database ---
<Sale(sales_person='Jehu Rudeforth', country='UK', product='Mint Chip Choco', date='2022-01-04', amount=$5,320 , boxes_shipped=180)>
<Sale(sales_person='Van Tuxwell', country='India', product='85% Dark Bars', date='2022-08-01', amount=$7,896 , boxes_shipped=94)>
<Sale(sales_person='Gigi Bohling', country='India', product='Peanut Butter Cubes', date='2022-07-07', amount=$4,501 , boxes_shipped=91)>
<Sale(sales_person='Jan Morforth', country='Australia', product='Peanut Butter Cubes', date='2022-04-27', amount=$12,726 , boxes_shipped=342)>
<Sale(sales_person='Jehu Rudeforth', country='UK', product='Peanut Butter Cubes', date='2022-02-24', amount=$13,685 , boxes_shipped=184)>
<Sale(sales_person='Van Tuxwell', country='India', product='Smooth Sliky Salty', date='2022-06-06', amount=$5,376 , boxes_shipped=38)>
<Sale(sales_per